In [1]:
# this is a cloud clustering assingment document 1

In [2]:
def GridRead(file,item):
    with open(file, 'r') as f:
        objGrid = ijson.items(f, item)
        colGrid = list(objGrid)
    return colGrid

In [3]:
def getGridDF(gridObj):
    grid_df = pd.DataFrame(columns=['id','xmin','xmax','ymin','ymax'],dtype='float')
    for row in gridObj:
        id = row["properties"]['id']
        xmin = row["properties"]['xmin']
        xmax = row["properties"]['xmax']
        ymin = row["properties"]['ymin']
        ymax = row["properties"]['ymax']
        row_df = pd.DataFrame([[id,xmin,xmax,ymin,ymax]],columns=['id','xmin','xmax','ymin','ymax'],dtype='float')
        grid_df = grid_df.append(row_df,ignore_index=True, sort=False)
    return grid_df

In [4]:
def getTweetDF(tweetObj):
    tweet_df = pd.DataFrame(columns=['id','x','y','hashlist'])
    for row in tweetObj:
        id = row["id"]
        coord0 = row["value"]['geometry']['coordinates'][0]
        coord1 = row["value"]['geometry']['coordinates'][1]
        hashtags = row['doc']['entities']['hashtags']
        hashlist = []
        for hashtg in hashtags:
            hashlist.append(hashtg['text'])
        row_df = pd.DataFrame([[id,coord0,coord1,hashlist]],columns=['id','x','y','hashlist'],dtype='float')
        row_df = row_df.dropna(axis=0)
        tweet_df = tweet_df.append(row_df,ignore_index=True,sort=False)
    return tweet_df

In [5]:
def locateBlock(x,y): 
    c = grid_df[(x >= grid_df['xmin']) &
            (x < grid_df['xmax']) &
            (y >= grid_df['ymin']) &
            (y < grid_df['ymax'])].id

    if len(c) == 0:
        return None
    else:
        return c.iat[0]

In [6]:
def locateBlock2(dpf): 
    c = grid_df[(dpf['x'] >= grid_df['xmin']) &
            (dpf['x'] < grid_df['xmax']) &
            (dpf['y'] >= grid_df['ymin']) &
            (dpf['y'] < grid_df['ymax'])].id

    if len(c) == 0:
        return None
    else:
        return c.iat[0]

In [7]:
import ijson
import pandas as pd
import numpy as np
import datetime
import multiprocessing as mp
fileGrid = "data/melbGrid.json"
readGrid = GridRead(fileGrid,'features.item')
grid_df = getGridDF(readGrid)
#print(grid_df)

In [8]:
fileTwitter = "data/smallTwitter.json"
rowTwit = GridRead(fileTwitter,'rows.item')
tweet_df = getTweetDF(rowTwit)
print(tweet_df.head(4))

             id           x          y     hashlist
0  5.703792e+17  144.923401 -37.959358  [melbourne]
1  5.703792e+17  144.923401 -37.959358  [melbourne]
2  5.703814e+17  144.923401 -37.959358  [melbourne]
3  5.703814e+17  144.923401 -37.959358  [melbourne]


In [9]:
#filter records with no coordinate information
#print(tweet_df.head(10))

In [10]:
# def parallelise(dataframe, func):
#     df_split = pd.DataFrame(np.array_split(dataframe, partitions))
#     pool = mp.Pool(cores)
#     df_return = pd.concat(pool.map(func,df_split),ignore_index=True)
#     pool.close()
#     return df_return

In [11]:
import os
import concurrent.futures

num_processes = mp.cpu_count()


# Process the rows in chunks in parallel
time1 = datetime.datetime.now()
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    tweet_df['GridCode'] = list(pool.map(locateBlock, tweet_df['x'],tweet_df['y'], chunksize=10)) # With a progressbar

time2 = datetime.datetime.now()
print("Time: Parallel execution of locate block = ",(time2-time1))
print(tweet_df.head(5))


Time: Parallel execution of locate block =  0:00:03.084119
             id           x          y     hashlist GridCode
0  5.703792e+17  144.923401 -37.959358  [melbourne]     None
1  5.703792e+17  144.923401 -37.959358  [melbourne]     None
2  5.703814e+17  144.923401 -37.959358  [melbourne]     None
3  5.703814e+17  144.923401 -37.959358  [melbourne]     None
4  5.703814e+17  144.923401 -37.959358  [melbourne]     None


In [12]:
time5 = datetime.datetime.now()
tweet_df['GridCode'] = tweet_df.apply(locateBlock2,axis=1)
time6 = datetime.datetime.now()
print("Time: Series execution of locate block", (time6-time5))
print(tweet_df.GridCode.value_counts())
#tweet_df.groupby(['GridCode']).size().sort_values(ascending=False)


Time: Series execution of locate block 0:00:14.715046
C2    3487
C3    3227
D3     281
Name: GridCode, dtype: int64


In [13]:
#proc_tweet_df = tweet_df.hashlist.apply(pd.Series).merge(tweet_df, left_index = True, right_index = True).drop(["hashlist"], axis = 1).melt(id_vars = ['id', 'x', 'y', 'GridCode'], value_name = "hashtag").drop("variable", axis = 1) .dropna()
#print(proc_tweet_df.head())

In [14]:
def explodeDF(tweet_df):
    result = tweet_df.set_index(['id', 'x','y','GridCode']).hashlist.apply(pd.Series).stack().reset_index(name = 'hashtag').drop('level_4', axis = 1)
    return(result)


In [15]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_processes)
    pool = Pool(num_processes)
    print("here")
    #print(pool.map(func, df_split))
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [16]:
from multiprocessing import Pool
time3 = datetime.datetime.now()
proc_tweet_df = parallelize_dataframe(tweet_df, explodeDF)
# with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
#      proc_tweet_df = list(pool.map(explodeDF, tweet_df, chunksize=10)) # With a progressbar

time4 = datetime.datetime.now()
print("Time: Parallel execution of explode",(time4-time3))
proc_tweet_df.groupby(['GridCode','hashtag']).size().sort_values(ascending=False).reset_index().groupby(['GridCode']).head(5).sort_values(['GridCode']).reset_index().drop(["index"], axis = 1)

#print(proc_tweet_df.head(10))

here
Time: Parallel execution of explode 0:00:00.793434


,GridCode,hashtag,0
0,C2,melbourne,215
1,C2,beach,116
2,C2,brightonbeach,110
3,C2,Melbourne,92
4,C2,australia,85
5,C3,melbourne,72
6,C3,brightonbeach,39
7,C3,brighton,38
8,C3,Melbourne,27
9,C3,beach,20


In [17]:
time7 = datetime.datetime.now()
proc_tweet_df2 = tweet_df.hashlist.apply(pd.Series).merge(tweet_df, left_index = True, right_index = True).drop(["hashlist"], axis = 1).melt(id_vars = ['id', 'x', 'y', 'GridCode'], value_name = "hashtag").drop("variable", axis = 1) .dropna()
time8 = datetime.datetime.now()

print("Time: Series execution of explode", (time8-time7))
proc_tweet_df2.groupby(['GridCode','hashtag']).size().sort_values(ascending=False).reset_index().groupby(['GridCode']).head(5).sort_values(['GridCode']).reset_index().drop(["index"], axis = 1)


Time: Series execution of explode 0:00:01.564529


,GridCode,hashtag,0
0,C2,melbourne,215
1,C2,beach,116
2,C2,brightonbeach,110
3,C2,Melbourne,92
4,C2,australia,85
5,C3,melbourne,72
6,C3,brightonbeach,39
7,C3,brighton,38
8,C3,Melbourne,27
9,C3,beach,20
